# Introduction à la La durée de vie

---

> Le concept de durée de vie ou lifetime en anglais définit le temps que vivra une variable dans le programme. Nous avons déjà constaté qu'une variable n'existait plus si on sortait du bloc dans lequel elle était déclarée et si elle n'est pas retournée par ce dernier :

In [3]:
fn main() {
    let a: usize = 42; // Si elle n'est pas shadowed, la variable a vivra durant toute la fonction main()
    {
        let b: usize = 21; // la variable b existera dans tout son sous-bloc
        {
            let c: usize = 11;
            dbg!(&c);
        }
        dbg!(&b); // J'ai encore acces a la variable b ici
    }
    // dbg(b) // ERREUR ! b n'existe plus en point du code
    dbg!(a); // A la fin du main(), j'ai toujours acces a la variable a
}
main()

[src/lib.rs:8] &c = 11
[src/lib.rs:10] &b = 21
[src/lib.rs:13] a = 42


()

In [4]:
fn main() {
    let a: usize = { // Ce bloc est une expresion et retourne la variable b
        let b: usize = 42;
        b
    };
    dbg!(a); // Ce qui etait la variable b vit toujours ici
}
main()

[src/lib.rs:7] a = 42


()

> Nous avons aussi entrevu dans le chapitre précédent qu'une référence n'existait plus à partir de sa dernière utilisation...

In [2]:
fn main() {
    let mut num = 4;
    let ref_num = &mut num;
    *ref_num += 1;
    *ref_num += 1;
    dbg!(ref_num); // ref_num ne vit plus apres cette instruction
    
    num += 1;
    dbg!(num);
}
main();

[src/lib.rs:7] ref_num = 6
[src/lib.rs:10] num = 7


> Il existe en Rust une notation spécifique aux durée de vie, elle peut être utilisée dans les fonctions, les méthodes ou les structures.

## Les durées de vie implicites :

**Les annotations de durées de vie concernent les références.**

> La plupart du temps, rust inférera les durées de vie sans que l'utilisateur n'ait à s'en soucier. Rappelons que la règle principale des durées de vie dit qu'un objet référencé DOIT vivre au moins aussi longtemps que les références sur celui-ci. C'est la garantie qu'il n'y aura jamais de référence Null ou invalides.

### Exemple avec une fonction

- Prenons par exemple ce bout de code :

In [13]:
fn first_element(array: &[u8; 3]) -> &u8 { // Cette fonction retourne une reference sur le premier element
    &array[0]
}
let v = [1, 2, 3];
dbg!(first_element(&v));

[src/lib.rs:115] first_element(&v) = 1


- Cette fonction retourne une référence sur le premier élément du tableau. Sachez que Rust, en interne, va la réécrire comme ci-dessous :

In [12]:
fn first_element<'a>(array: &'a [u8; 3]) -> &'a u8 { // Ajout de 'a par le compilateur
    &array[0]
}
let v = [1, 2, 3];
dbg!(first_element(&v));

[src/lib.rs:115] first_element(&v) = 1


> Une annotation de la durée de vie `'a` est ajoutée à l'entrée et a la sortie de la fonction. Le choix de la lettre a est parfaitement arbitraire, b irait aussi bien.

> L'annotation de durée de vie doit toujours être déclarée avant d'être utilisée `<'a>`.

> L'emploi de ce lifetime signifie que **L’entrée doit vivre au moins aussi longtemps que la sortie**. Et ceci est logique, car, après tout, la fonction first_element retourne une référence sur un élément déjà passé par référence, donc **la référence sur l'élément du tableau NE SERAIT PLUS VALIDE si le tableau entier venait à disparaître !**. D'où le fait que l'entrée doit vivre au moins aussi longtemps que la sortie.

> **En Rust, il n'est pas possible d'avoir des références qui pointent sur quelque chose qui n'existe plus !**

### Exemple avec une structure

- Si on prend l'exemple d'une structure :

In [7]:
{
    struct MaStruct {
        inner_a: u32,
        inner_b: u32,
    }
    fn get_inner_a(t: &MaStruct) -> &u32 {
        &t.inner_a
    }
    fn get_inner_a_by_rust<'a>(t: &'a MaStruct) -> &'a u32 { // Infered lifetime
        &t.inner_a
    }
}

()

> Les fonctions get_inner_a et get_inner_a_by_rust sont strictement les mêmes.

 - Les mêmes règles s'appliqueront dans le cas d'une méthode d'une structure (&self ou μt self) retournant une référence sur un champ de cette dernière. La structure en entrée doit vivre au moins aussi longtemps que la référence en sortie :

In [8]:
{
    struct MaStruct {
        inner_a: u32,
        inner_b: u32,
    }
    impl MaStruct {
        fn get_inner_a(&self) -> &u32 {
            &self.inner_a
        }
    }
}

()

... sera remplacée par :
```
impl MaStruct {
    fn get_inner_a<'a>(&'a self) -> &'a u32 {
        &self.inner_a
    }
}
```

> **NB** Les annotations de durée de vie telles `'a` doivent toujours être déclarées après le nom de la fonction et avant les parenthèses d'où le `<'a>`.

## Quand il faut être explicite

### Les structures et les énumérations

- Quand une structure ou une énumération contient une référence sur quelque chose, l'annotation du lifetime y est obligatoire :

In [15]:
{
    #[derive(Debug)]
    enum Famille {
        Fruit,
        Cereale,
    }

    #[derive(Debug)]
    struct Aliment<'a> { // Puisqu'il y a une reference, je met une annotation de lifetime
        calories: usize,
        toxicity: bool,
        famille: &'a Famille, // La variable de type Famille referencee ici doit vivre au moins aussi longtemps 
                              // que la structure Aliment.
    }

    let fruit = Famille::Fruit;
    let poire = Aliment {
        calories: 100,
        toxicity: false,
        famille: &fruit,
    };
    dbg!(poire);
}

[src/lib.rs:68] poire = Aliment {
    calories: 100,
    toxicity: false,
    famille: Fruit,
}


()

### Les problèmes de Lifetime

- Dans certains cas de figure, il est obligatoire de spécifier les durées de vie. C'est par exemple le cas d'une fonction qui prend 2 références en entrée, et une en sortie, Rust ne saura pas comment déterminer quelles durées de vie doivent être liées et exigera que l'on annote manuellement les durées de vie :

In [12]:
{
    struct MaStruct {
        inner_a: u32,
        inner_b: u32,
    }
    impl MaStruct {
        fn get_inner_a(&self) -> &u32 {
            &self.inner_a
        }
    }
    fn dump_a_string_and_get_inner_a(t: &MaStruct, s: &str) -> &u32 {
        println!("{}", s);
        &t.inner_a
    }
}

Error: missing lifetime specifier

- Le compilateur suggère de définir la fonction ainsi : **dump_a_string_and_get_inner_a\<'a>(t: &'a MaStruct, s: &'a str) -> &'a u32**. Appliquons la modification :

In [11]:
{
    struct MaStruct {
        inner_a: u32,
        inner_b: u32,
    }
    impl MaStruct {
        fn get_inner_a(&self) -> &u32 {
            &self.inner_a
        }
    }
    fn dump_a_string_and_get_inner_a<'a>(t: &'a MaStruct, s: &'a str) -> &'a u32 {
        println!("{}", s);
        &t.inner_a
    }
}
// Ce code compile

- Maintenant, utilisons la fonction dans ce bout de code :

In [16]:
{
    struct MaStruct {
        inner_a: u32,
        inner_b: u32,
    }
    impl MaStruct {
        fn get_inner_a(&self) -> &u32 {
            &self.inner_a
        }
    }
    fn dump_a_string_and_get_inner_a<'a>(t: &'a MaStruct, s: &'a str) -> &'a u32 {
        println!("{}", s);
        &t.inner_a
    }

    fn get_output(t: &MaStruct) -> &u32 {
        let s = String::from("Toto");
        let output = dump_a_string_and_get_inner_a(&t, &s);
        output
    }
    let t = MaStruct {
        inner_a: 1,
        inner_b: 2,
    };
    dbg!(get_output(&t));
}

Error: cannot return value referencing local variable `s`

> Ce code ne compile hélas pas. Le compilateur nous indique qu'une référence de la chaîne de caractère s a été empruntée par la fonction dump_a_string_and_get_inner_a() et qu'elle est liée a son retour, ici output... Hors la variable s est déjà détruite au moment où l'on tente de lire le retour de la fonction get_output() via la macro dbg! a la fin.

> Le problème vient de notre écriture du prototype de la fonction dump_a_string_and_get_inner_a. `dump_a_string_and_get_inner_a<'a>(t: &'a MaStruct, s: &'a str) -> &'a u32`. On mettant la même annotation de lifetime sur les deux entrées et la sortie comme le suggérait le compilateur, nous indiquons que les deux références en entrée doivent vivre au moins aussi longtemps que la sortie. Or, ce n'est pas le cas dans notre exemple, puisque **la chaîne s vit moins longtemps !**

- Afin de régler le problème de compilation, nous devons rajouter une autre annotation de lifetime, ici notée `'b` pour notre référence sur la chaîne de caractère. On indique ainsi au compilateur que la durée de vie du paramètre d'entrée s, notre chaîne de caractère, n'est pas liée à la durée de vie de la sortie de la fonction :

In [17]:
{
    struct MaStruct {
        inner_a: u32,
        inner_b: u32,
    }
    impl MaStruct {
        fn get_inner_a(&self) -> &u32 {
            &self.inner_a
        }
    }
    fn dump_a_string_and_get_inner_a<'a, 'b>(t: &'a MaStruct, s: &'b str) -> &'a u32 { // Ajout du lifetime 'b
        println!("{}", s);
        &t.inner_a
    }

    fn get_output(t: &MaStruct) -> &u32 {
        let s = String::from("Toto");
        let output = dump_a_string_and_get_inner_a(&t, &s);
        output
    }
    let t = MaStruct {
        inner_a: 1,
        inner_b: 2,
    };
    dbg!(get_output(&t));
}

Toto


[src/lib.rs:71] get_output(&t) = 1


()

> **NB** Ici deux lifetimes doivent être déclarés. `<'a, 'b>`

> Le prototype final sera `dump_a_string_and_get_inner_a<'a, 'b>(t: &'a MaStruct, s: &'b str) -> &'a u32`

### Des contraintes supplementaires

> Enfin. Il est aussi possible de spécifier une hiérarchie entre la durée de vie des annotations de lifetime.

- Ici, 'a doit vivre plus longtemps que 'b :

```
fn test<'a: 'b, 'a>(t: &'a str, s: &'b str) -> &'a str
```

- Ici, 'b doit vivre plus longtemps que 'a :

```
fn test<'a, 'b: 'a>(t: &'a str, s: &'b str) -> &'a str
```

## Le lifetime static

- Il existe un lifetime spécial dont la particularité est d'indiquer au compilateur que la variable référencée vit pendant toute la durée du programme. On le nomme le lifetime **'static**. Il est fréquemment utilisé avec les chaînes de caractère :

In [30]:
const LIVRE: &str = "Les fleurs bleues";

fn main() {
    fn get_book() -> &'static str {
        &LIVRE
    }
    dbg!(get_book());
}
main()

[src/lib.rs:9] get_book() = "Les fleurs bleues"


()

- La compilation s'avérera impossible si la variable referencee ne vit pas durant tout le programme :

In [32]:
fn main() {
    let livre: &str = "Les fleurs bleues";
    fn get_book<'a>(livre: &'a str) -> &'static str {
        &livre
    }
    dbg!(get_book(&livre));
}
main()

Error: lifetime may not live long enough

## Exercice

### Corriger les annotations de lifetime de la fonction generate_struct()

In [44]:
fn main() {
    #[derive(Debug)]
    struct S<'a> {
        s1: &'a str,
        s2: &'a str,
    }
    fn generate_struct<'a, 'b>(chaine_1: &'a str, chaine_2: &'b str, chaine_3: &'a str) -> S<'b> {
        println!("{}", chaine_2);
        S {
            s1: chaine_1,
            s2: chaine_3,
        }
    }
    fn intermediate_fn<'a>(chaine_1: &'a str, chaine_3: &'a str) -> S<'a> {
        let chaine_2 = "La chaine numero 2";
        generate_struct(chaine_1, chaine_2, chaine_3)
    }
    let chaine_3 = "La chaine numero 3";
    let chaine_1 = "La chaine numero 1";
    
    let s = intermediate_fn(chaine_1, chaine_3);
    println!("{:?}", s);
}
main()

Error: lifetime may not live long enough

Il y a deux solutions à ce problème :
> ```
> fn generate_struct<'a, 'b>(chaine_1: &'a str, chaine_2: &'b str, chaine_3: &'a str) -> S<'a>
> ```
> **Il semblait évident de relier le retour de la fonction au lifetime 'a plutôt que 'b. Cette solution fonctionne ...**

ou

> ```
> fn generate_struct<'a>(chaine_1: &'a str, chaine_2: &'a str, chaine_3: &'a str) -> S<'a>
> ```
> **... pourtant cet exemple n'est pas tout à fait le même que celui que nous avons vu plus haut, la fonction ne retourne pas de référence de type structure S mais une instance réelle d'une structure S, ainsi avec l'utilisation unique d'un seul lifetime pour toutes les références, les règles de Rust sont respectees. Cette solution fonctionne donc aussi.**

*Notons enfin que le compilateur nous force à donner une annotation de lifetime pour le type structure S.*

### Quelqu'un m'a dit que ce code est juste, pourquoi a-t-il raison ?

```
{
    #[derive(Debug)]
    struct A<'a> {
        i: &'a i32,
    }
    

    fn main() {
        let r: A;
        {
            let local: i32 = 42;
            r = A{
                i: &local
            };
        }
        dbg!(r);
    }
}
```

> Et pourtant... Ici, on voit que la variable "local" qui est referencee par la structure "r" ne vit pas assez longtemps ! Au moment du dbg! final, la variable "local" ne serait plus sur la pile, on est sorti de son scope... Bref, ça ne compile pas.
```
[E0597] Error: `local` does not live long enough
    ╭─[command_45:1:1]
    │
 13 │                 i: &local
    ·                    ─┬  
    ·                     ╰── borrowed value does not live long enough
    · 
 15 │         }
    ·         ┬  
    ·         ╰── `local` dropped here while still borrowed
 16 │         dbg!(r);
    ·              ┬  
    ·              ╰── borrow later used here
────╯


```